## DSA Competition - June 2019
Developer: Cristina Santana Souza

Linkedin: https://www.linkedin.com/in/cristina-santana-souza/

Feature Engineering: https://github.com/crisansou/kaggle/tree/master/dsajunho

*This kernel was executed only for 1000 records, to see the full execution check link in Github

## Imports

In [1]:
import datetime as dt
import pandas as pd
import pandas_profiling as pf
import warnings
warnings.filterwarnings('ignore')

#To suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 100)

## Load datasets

In [2]:
## Path
path_train = "dataset_treino.csv"
path_test = "dataset_teste.csv"
path_merchants = "comerciantes.csv"
path_new_trs = "novas_transacoes_comerciantes.csv"
path_hst_trs = "transacoes_historicas.csv"

## Read

### Train
parse_dates_train_test = ["first_active_month"]

dtype_train = {'card_id'  : 'category',
               'feature_1': 'category',
               'feature_2': 'category',
               'feature_3': 'category',
               'target'   : 'float64'}

train_df = pd.read_csv(path_train, encoding="utf-8", dtype=dtype_train, parse_dates=parse_dates_train_test)

### Test
dtype_test = {'card_id'  : 'category',
              'feature_1': 'category',
              'feature_2': 'category',
              'feature_3': 'category'}

test_df = pd.read_csv(path_test, encoding="utf-8", dtype=dtype_test, parse_dates=parse_dates_train_test)

### Merchants
dtype_merchants = {'merchant_id'                : 'category',
                   'merchant_group_id'          : 'category',
                   'merchant_category_id'       : 'category',
                   'subsector_id'               : 'category',
                   'numerical_1'                : 'float64',
                   'numerical_2'                : 'float64',
                   'category_1'                 : 'category',
                   'most_recent_sales_range'    : 'category',
                   'most_recent_purchases_range': 'category',
                   'avg_sales_lag3'             : 'float64',
                   'avg_purchases_lag3'         : 'float64',
                   'active_months_lag3'         : 'int64',
                   'avg_sales_lag6'             : 'float64',
                   'avg_purchases_lag6'         : 'float64',
                   'active_months_lag6'         : 'int64',
                   'avg_sales_lag12'            : 'float64',
                   'avg_purchases_lag12'        : 'float64',
                   'active_months_lag12'        : 'int64',
                   'category_4'                 : 'category',
                   'city_id'                    : 'category',
                   'state_id'                   : 'category',                   
                   'category_2'                 : 'category'}

merchants_df = pd.read_csv(path_merchants, encoding="utf-8", dtype=dtype_merchants)

### New Merchant Transactions
parse_dates_trs = ["purchase_date"]

dtype_new_trs = {'authorized_flag'     : 'category',
                 'card_id'             : 'category',
                 'city_id'             : 'category',
                 'category_1'          : 'category',
                 'installments'        : 'int64',
                 'category_3'          : 'category',
                 'merchant_category_id': 'category',
                 'merchant_id'         : 'category',
                 'month_lag'           : 'int64',
                 'purchase_amount'     : 'float64',
                 'category_2'          : 'float64',
                 'state_id'            : 'category',
                 'subsector_id'        : 'category'}

new_trs_df = pd.read_csv(path_new_trs, encoding="utf-8", dtype=dtype_new_trs, parse_dates=parse_dates_trs)
#new_trs_df = pd.read_csv(path_new_trs, encoding="utf-8", dtype=dtype_new_trs, parse_dates=parse_dates_trs, nrows = 1000)

### Historical Transactions
hst_trs_df = pd.read_csv(path_hst_trs, encoding="utf-8", dtype=dtype_new_trs, parse_dates=parse_dates_trs)
#hst_trs_df = pd.read_csv(path_hst_trs, encoding="utf-8", dtype=dtype_new_trs, parse_dates=parse_dates_trs, nrows = 1000)

## List brazilian holidays from 2011 to 2018

http://www.anbima.com.br/feriados/feriados.asp

In [3]:
lst_holidays = ['2011-01-01','2011-03-07','2011-03-08','2011-04-21','2011-04-22','2011-05-01','2011-06-23','2011-09-07','2011-10-12','2011-11-02','2011-11-15','2011-12-25','2012-01-01','2012-02-20', '2012-02-21','2012-04-06','2012-04-21','2012-05-01','2012-06-07','2012-09-07','2012-10-12','2012-11-02','2012-11-15','2012-12-25','2013-01-01','2013-02-11','2013-02-12','2013-03-29','2013-04-21','2013-05-01','2013-05-30','2013-09-07','2013-10-12','2013-11-02','2013-11-15','2013-12-25','2014-01-01','2014-03-03','2014-03-04','2014-04-18','2014-04-21','2014-05-01','2014-06-19','2014-09-07','2014-10-12','2014-11-02','2014-11-15','2014-12-25','2015-01-01','2015-02-16','2015-02-17','2015-04-03','2015-04-21','2015-05-01','2015-06-04','2015-09-07','2015-10-12','2015-11-02','2015-11-15','2015-12-25','2016-01-01','2016-02-08','2016-02-09','2016-03-25','2016-04-21','2016-05-01','2016-05-26','2016-09-07','2016-10-12','2016-11-02','2016-11-15','2016-12-25','2017-01-01','2017-02-27','2017-02-28','2017-04-14','2017-04-21','2017-05-01','2017-06-15','2017-09-07','2017-10-12','2017-11-02','2017-11-15','2017-12-25','2018-01-01','2018-02-12','2018-02-13','2018-03-30','2018-04-21','2018-05-01','2018-05-31','2018-09-07','2018-10-12','2018-11-02','2018-11-15','2018-12-25']

## List commercial holidays from 2011 to 2018

In [4]:
lst_black_friday = ['2018-11-23','2017-11-24','2016-11-25','2015-11-27','2014-11-28','2013-11-29','2012-11-23','2011-11-25']
lst_mothers_day = ['2018-05-13','2017-05-14','2016-05-08','2015-05-10','2014-05-11','2013-05-12','2012-05-13','2011-05-08']
lst_fathers_day = ['2018-08-12','2017-08-13','2016-08-14','2015-08-09','2014-08-10','2013-08-11','2012-08-12','2011-08-14']
lst_childrens_day = ['2018-10-12','2017-10-12','2016-10-12','2015-10-12','2014-10-12','2013-10-12','2012-10-12','2011-10-12']
lst_valentines_day = ['2018-06-12','2017-06-12','2016-06-12','2015-06-12','2014-06-12','2013-06-12','2012-06-12','2011-06-12']

## Fill null field of test file - min purchase date

In [5]:
test_df[test_df.first_active_month.isnull()]

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaT,C_ID_c27b4f80f7,5,2,1


In [6]:
hst_trs_df[hst_trs_df.card_id == "C_ID_c27b4f80f7"].purchase_date.min()

Timestamp('2017-03-09 01:34:22')

In [7]:
test_df['first_active_month'] = pd.to_datetime(test_df['first_active_month'].fillna('2017-03-09'))
test_df[test_df.card_id == "C_ID_c27b4f80f7"]

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,2017-03-09,C_ID_c27b4f80f7,5,2,1


## Makes common transformations to training and test datasets

In [8]:
for df in [train_df,test_df]:
    df['first_active_year'] = df['first_active_month'].dt.year
    df['first_active_months'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (dt.datetime.today() - df['first_active_month']).dt.days
    df['first_active_quarter'] = df['first_active_month'].dt.quarter

In [9]:
train_df.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_year,first_active_months,elapsed_time,first_active_quarter
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820,2017,6,758,2
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.393,2017,1,909,1
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688,2016,8,1062,3


In [10]:
test_df.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,first_active_year,first_active_months,elapsed_time,first_active_quarter
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,2017,4,819,2
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,2017,1,909,1
2,2017-08-01,C_ID_b709037bc5,5,1,1,2017,8,697,3


In [11]:
del train_df['first_active_month']
del test_df['first_active_month']

## Fill some NAs

In [12]:
new_trs_df['category_2'].astype(float).fillna(1.0,inplace=True)
new_trs_df['category_3'].fillna('A',inplace=True)
hst_trs_df['category_2'].astype(float).fillna(1.0,inplace=True)
hst_trs_df['category_3'].fillna('A',inplace=True)

## Divide the history table by authorized and unauthorized transaction

In [13]:
aut_hst_trs_df = hst_trs_df[hst_trs_df['authorized_flag'] == 'Y'].copy().reset_index(drop=True)
rej_hst_trs_df = hst_trs_df[hst_trs_df['authorized_flag'] == 'N'].copy().reset_index(drop=True)

In [14]:
print("hst_trs_df.shape: ",hst_trs_df.shape)
print("aut_hst_trs_df.shape: ",aut_hst_trs_df.shape)
print("rej_hst_trs_df.shape: ",rej_hst_trs_df.shape)
del hst_trs_df

hst_trs_df.shape:  (29112361, 14)
aut_hst_trs_df.shape:  (26595452, 14)
rej_hst_trs_df.shape:  (2516909, 14)


## Makes common transformations to transaction datasets

In [15]:
for df in [new_trs_df, aut_hst_trs_df, rej_hst_trs_df]:
    df['purchase_year'] = df['purchase_date'].dt.year
    df['purchase_week_year'] = df['purchase_date'].dt.weekofyear
    df['purchase_month'] = df['purchase_date'].dt.month
    df['purchase_weekday'] = df['purchase_date'].dt.weekday
    df['purchase_weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['price'] = df['purchase_amount'] / df['installments']
    df['purchase_month_diff'] = (((dt.datetime.today() - df['purchase_date']).dt.days)//30) + df['month_lag']
    df['category_1'] = df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_3'] = df['category_3'].map({'A':0, 'B':1, 'C':2}).astype(int)
    df['category_2_purchase'] = df.groupby(['category_2'])['purchase_amount'].transform('mean')
    df['category_3_purchase'] = df.groupby(['category_3'])['purchase_amount'].transform('mean')

#### Holidays

In [16]:
new_trs_df['near_holidays'] = 0
aut_hst_trs_df['near_holidays'] = 0
rej_hst_trs_df['near_holidays'] = 0
for i in lst_holidays:
    new_trs_df['near_holidays'] = (pd.to_datetime(i) - new_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + new_trs_df['near_holidays']
    aut_hst_trs_df['near_holidays'] = (pd.to_datetime(i) - aut_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + aut_hst_trs_df['near_holidays']
    rej_hst_trs_df['near_holidays'] = (pd.to_datetime(i) - rej_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + rej_hst_trs_df['near_holidays']


#### Black Friday

In [17]:
new_trs_df['near_black_friday'] = 0
aut_hst_trs_df['near_black_friday'] = 0
rej_hst_trs_df['near_black_friday'] = 0
for i in lst_black_friday:
    new_trs_df['near_black_friday'] = (pd.to_datetime(i) - new_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + new_trs_df['near_black_friday']
    aut_hst_trs_df['near_black_friday'] = (pd.to_datetime(i) - aut_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + aut_hst_trs_df['near_black_friday']
    rej_hst_trs_df['near_black_friday'] = (pd.to_datetime(i) - rej_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + rej_hst_trs_df['near_black_friday']


#### Mother's Day

In [18]:
new_trs_df['near_mothers_day'] = 0
aut_hst_trs_df['near_mothers_day'] = 0
rej_hst_trs_df['near_mothers_day'] = 0
for i in lst_mothers_day :
    new_trs_df['near_mothers_day'] = (pd.to_datetime(i) - new_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + new_trs_df['near_mothers_day']
    aut_hst_trs_df['near_mothers_day'] = (pd.to_datetime(i) - aut_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + aut_hst_trs_df['near_mothers_day']
    rej_hst_trs_df['near_mothers_day'] = (pd.to_datetime(i) - rej_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + rej_hst_trs_df['near_mothers_day']


#### Father's Day

In [19]:
new_trs_df['near_fathers_day'] = 0
aut_hst_trs_df['near_fathers_day'] = 0
rej_hst_trs_df['near_fathers_day'] = 0
for i in lst_fathers_day:
    new_trs_df['near_fathers_day'] = (pd.to_datetime(i) - new_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + new_trs_df['near_fathers_day']
    aut_hst_trs_df['near_fathers_day'] = (pd.to_datetime(i) - aut_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + aut_hst_trs_df['near_fathers_day']
    rej_hst_trs_df['near_fathers_day'] = (pd.to_datetime(i) - rej_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + rej_hst_trs_df['near_fathers_day']


#### Children's Day

In [20]:
new_trs_df['near_childrens_day'] = 0
aut_hst_trs_df['near_childrens_day'] = 0
rej_hst_trs_df['near_childrens_day'] = 0
for i in lst_childrens_day:
    new_trs_df['near_childrens_day'] = (pd.to_datetime(i) - new_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + new_trs_df['near_childrens_day']
    aut_hst_trs_df['near_childrens_day'] = (pd.to_datetime(i) - aut_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + aut_hst_trs_df['near_childrens_day']
    rej_hst_trs_df['near_childrens_day'] = (pd.to_datetime(i) - rej_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + rej_hst_trs_df['near_childrens_day']


#### Valentine's Day

In [21]:
new_trs_df['near_valentines_day'] = 0
aut_hst_trs_df['near_valentines_day'] = 0
rej_hst_trs_df['near_valentines_day'] = 0
for i in lst_valentines_day:
    new_trs_df['near_valentines_day'] = (pd.to_datetime(i) - new_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + new_trs_df['near_valentines_day']
    aut_hst_trs_df['near_valentines_day'] = (pd.to_datetime(i) - aut_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + aut_hst_trs_df['near_valentines_day']
    rej_hst_trs_df['near_valentines_day'] = (pd.to_datetime(i) - rej_hst_trs_df['purchase_date']).dt.days.apply(lambda x: 1 if x > 0 and x <= 30 else 0) + rej_hst_trs_df['near_valentines_day']


In [22]:
new_trs_df.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_year,purchase_week_year,purchase_month,purchase_weekday,purchase_weekend,price,purchase_month_diff,category_2_purchase,category_3_purchase,near_holidays,near_black_friday,near_mothers_day,near_fathers_day,near_childrens_day,near_valentines_day
0,Y,C_ID_415bb3a509,107,0,1,1,307,M_ID_b0c793002c,1,-0.558,2018-03-11 14:57:36,1.000,9,19,2018,10,3,6,1,-0.558,16,-0.569,-0.606,1,0,0,0,0,0
1,Y,C_ID_415bb3a509,140,0,1,1,307,M_ID_88920c89e8,1,-0.570,2018-03-19 18:53:37,1.000,9,19,2018,12,3,0,0,-0.570,16,-0.569,-0.606,1,0,0,0,0,0
2,Y,C_ID_415bb3a509,330,0,1,1,507,M_ID_ad5237ef6b,2,-0.551,2018-04-26 14:08:44,1.000,9,14,2018,17,4,3,0,-0.551,16,-0.569,-0.606,1,0,1,0,0,0


In [23]:
aut_hst_trs_df.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_year,purchase_week_year,purchase_month,purchase_weekday,purchase_weekend,price,purchase_month_diff,category_2_purchase,category_3_purchase,near_holidays,near_black_friday,near_mothers_day,near_fathers_day,near_childrens_day,near_valentines_day
0,Y,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_e020e9b302,-8,-0.703,2017-06-25 15:33:07,1.000,16,37,2017,25,6,6,1,-inf,16,-0.606,-0.634,0,0,0,0,0,0
1,Y,C_ID_4e6213e9bc,88,0,0,0,367,M_ID_86ec983688,-7,-0.733,2017-07-15 12:10:45,1.000,16,16,2017,28,7,5,1,-inf,16,-0.606,-0.634,0,0,0,1,0,0
2,Y,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_979ed661fc,-6,-0.720,2017-08-09 22:04:29,1.000,16,37,2017,32,8,2,0,-inf,16,-0.606,-0.634,1,0,0,1,0,0


In [24]:
rej_hst_trs_df.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_year,purchase_week_year,purchase_month,purchase_weekday,purchase_weekend,price,purchase_month_diff,category_2_purchase,category_3_purchase,near_holidays,near_black_friday,near_mothers_day,near_fathers_day,near_childrens_day,near_valentines_day
0,N,C_ID_4e6213e9bc,88,0,0,0,842,M_ID_22c9cfa265,-10,-0.730,2017-04-07 12:58:09,1.000,16,37,2017,14,4,4,0,-inf,17,9.817,13.068,3,0,0,0,0,0
1,N,C_ID_4e6213e9bc,88,0,0,0,367,M_ID_86ec983688,-5,-0.724,2017-09-17 22:40:27,1.000,16,16,2017,37,9,6,1,-inf,16,9.817,13.068,1,0,0,0,1,0
2,N,C_ID_4e6213e9bc,88,0,0,0,367,M_ID_86ec983688,-5,-0.724,2017-09-17 22:40:26,1.000,16,16,2017,37,9,6,1,-inf,16,9.817,13.068,1,0,0,0,1,0


## Creates dictionary to store the columns to be transformed

In [25]:
agg_cols = {}

agg_cols['purchase_year'] = ['nunique', 'mean', 'min', 'max']
agg_cols['purchase_week_year'] = ['nunique', 'mean', 'min', 'max']
agg_cols['purchase_month'] = ['nunique', 'mean', 'min', 'max']
agg_cols['purchase_weekday'] = ['nunique', 'mean', 'min', 'max']
agg_cols['purchase_weekend'] = ['nunique', 'mean', 'min', 'max']
agg_cols['purchase_amount'] = ['sum','max','min','mean']
agg_cols['installments'] = ['sum','max','mean','var']
agg_cols['purchase_date'] = ['max','min']
agg_cols['month_lag'] = ['max','min','mean']
agg_cols['merchant_id'] = ['nunique']
agg_cols['merchant_category_id'] = ['nunique']
agg_cols['near_holidays'] = ['sum','mean']
agg_cols['near_black_friday'] = ['sum','mean']
agg_cols['near_mothers_day'] = ['sum','mean']
agg_cols['near_fathers_day'] = ['sum','mean']
agg_cols['near_childrens_day'] = ['sum','mean']
agg_cols['near_valentines_day'] = ['sum','mean']
agg_cols['purchase_month_diff'] = ['mean', 'min', 'max', 'var']
agg_cols['price'] = ['sum','mean','max','min','var']
agg_cols["card_id"] = ['count']
agg_cols['category_1'] = ['mean','sum']
agg_cols['category_2'] = ['mean','sum']
agg_cols['category_3'] = ['mean','sum']
agg_cols['category_2_purchase'] = ['mean']
agg_cols['category_3_purchase'] = ['mean']

agg_cols

{'purchase_year': ['nunique', 'mean', 'min', 'max'],
 'purchase_week_year': ['nunique', 'mean', 'min', 'max'],
 'purchase_month': ['nunique', 'mean', 'min', 'max'],
 'purchase_weekday': ['nunique', 'mean', 'min', 'max'],
 'purchase_weekend': ['nunique', 'mean', 'min', 'max'],
 'purchase_amount': ['sum', 'max', 'min', 'mean'],
 'installments': ['sum', 'max', 'mean', 'var'],
 'purchase_date': ['max', 'min'],
 'month_lag': ['max', 'min', 'mean'],
 'merchant_id': ['nunique'],
 'merchant_category_id': ['nunique'],
 'near_holidays': ['sum', 'mean'],
 'near_black_friday': ['sum', 'mean'],
 'near_mothers_day': ['sum', 'mean'],
 'near_fathers_day': ['sum', 'mean'],
 'near_childrens_day': ['sum', 'mean'],
 'near_valentines_day': ['sum', 'mean'],
 'purchase_month_diff': ['mean', 'min', 'max', 'var'],
 'price': ['sum', 'mean', 'max', 'min', 'var'],
 'card_id': ['count'],
 'category_1': ['mean', 'sum'],
 'category_2': ['mean', 'sum'],
 'category_3': ['mean', 'sum'],
 'category_2_purchase': ['mean']

## Creates aggregation columns in transaction dataframes

In [26]:
agg_new_trs_df = new_trs_df.reset_index().groupby('card_id').agg(agg_cols)
agg_aut_hst_trs_df = aut_hst_trs_df.reset_index().groupby('card_id').agg(agg_cols)
agg_rej_hst_trs_df = rej_hst_trs_df.reset_index().groupby('card_id').agg(agg_cols)

In [27]:
# change column name
agg_new_trs_df.columns = pd.Index(['new_' + x[0] + '_' + x[1] for x in agg_new_trs_df.columns.tolist()])
agg_new_trs_df = agg_new_trs_df.reset_index()

agg_aut_hst_trs_df.columns = pd.Index(['aut_' + x[0] + '_' + x[1] for x in agg_aut_hst_trs_df.columns.tolist()])
agg_aut_hst_trs_df = agg_aut_hst_trs_df.reset_index()

agg_rej_hst_trs_df.columns = pd.Index(['rej_' + x[0] + '_' + x[1] for x in agg_rej_hst_trs_df.columns.tolist()])
agg_rej_hst_trs_df = agg_rej_hst_trs_df.reset_index()

## Create other columns after aggregation

In [28]:
city_new_trs_df = new_trs_df.groupby('card_id')['city_id'].value_counts().groupby(level=0).head(1).to_frame('qtde').reset_index()

city_new_trs_df.head(3)

,card_id,city_id,qtde
0,C_ID_0001238066,314,7
1,C_ID_0001793786,69,15
2,C_ID_0009150c11,17,8


In [29]:
city_aut_hst_trs_df = aut_hst_trs_df.groupby('card_id')['city_id'].value_counts().groupby(level=0).head(1).to_frame('qtde').reset_index()

city_aut_hst_trs_df.head(3)

,card_id,city_id,qtde
0,C_ID_0046db9f8a,110,102
1,C_ID_011b0d9794,-1,27
2,C_ID_01904d743d,314,40


In [34]:
del city_new_trs_df['qtde']
del city_aut_hst_trs_df['qtde']

In [35]:
agg_new_trs_df['new_elapsed_time_min_purchase'] = (dt.datetime.today() - agg_new_trs_df['new_purchase_date_min']).dt.days
agg_new_trs_df['new_elapsed_time_max_purchase'] = (dt.datetime.today() - agg_new_trs_df['new_purchase_date_max']).dt.days
agg_new_trs_df['new_purchase_date_diff'] = (agg_new_trs_df['new_purchase_date_max'] - agg_new_trs_df['new_purchase_date_min']).dt.days
agg_new_trs_df['new_purchase_date_avg'] = agg_new_trs_df['new_purchase_date_diff'] / agg_new_trs_df['new_card_id_count']
agg_new_trs_df = pd.merge(agg_new_trs_df, city_new_trs_df, on='card_id', how='left')
agg_new_trs_df.rename(columns={'city_id':'new_city_id'}, inplace=True)


agg_new_trs_df.head(3)

,card_id,new_purchase_year_nunique,new_purchase_year_mean,new_purchase_year_min,new_purchase_year_max,new_purchase_week_year_nunique,new_purchase_week_year_mean,new_purchase_week_year_min,new_purchase_week_year_max,new_purchase_month_nunique,new_purchase_month_mean,new_purchase_month_min,new_purchase_month_max,new_purchase_weekday_nunique,new_purchase_weekday_mean,new_purchase_weekday_min,new_purchase_weekday_max,new_purchase_weekend_nunique,new_purchase_weekend_mean,new_purchase_weekend_min,new_purchase_weekend_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_purchase_date_max,new_purchase_date_min,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_near_holidays_sum,new_near_holidays_mean,new_near_black_friday_sum,new_near_black_friday_mean,new_near_mothers_day_sum,new_near_mothers_day_mean,new_near_fathers_day_sum,new_near_fathers_day_mean,new_near_childrens_day_sum,new_near_childrens_day_mean,new_near_valentines_day_sum,new_near_valentines_day_mean,new_purchase_month_diff_mean,new_purchase_month_diff_min,new_purchase_month_diff_max,new_purchase_month_diff_var,new_price_sum,new_price_mean,new_price_max,new_price_min,new_price_var,new_card_id_count,new_category_1_mean,new_category_1_sum,new_category_2_mean,new_category_2_sum,new_category_3_mean,new_category_3_sum,new_category_2_purchase_mean,new_category_3_purchase_mean,new_elapsed_time_min_purchase,new_elapsed_time_max_purchase,new_purchase_date_diff,new_purchase_date_avg,new_city_id
0,C_ID_0001238066,1,2018.000,2018,2018,9,12.846,9,18,2,3.346,3,4,6,4.038,0,6,2,0.462,0,1,-14.850,-0.078,-0.741,-0.571,42,10,1.615,4.326,2018-04-30 19:57:30,2018-03-01 16:48:27,2,1,1.346,25,15,35,1.346,0,0.000,7,0.269,0,0.000,0,0.000,0,0.000,16.154,16,17,0.135,-13.283,-0.511,0.152,-0.741,0.068,26,0.077,2,1.522,35.000,1.115,29,-0.567,-0.507,485,424,60,2.308,314
1,C_ID_0001793786,1,2017.000,2017,2017,6,48.387,46,52,2,11.323,11,12,6,3.581,0,6,2,0.452,0,1,-0.230,3.130,-0.738,-0.007,0,0,0.000,0.000,2017-12-31 17:35:56,2017-11-15 15:44:20,2,1,1.323,31,21,24,0.774,6,0.194,0,0.000,0,0.000,0,0.000,0,0.000,20.000,20,20,0.000,nan,nan,inf,-inf,nan,31,0.000,0,1.759,51.000,0.000,0,-0.562,-0.593,591,545,46,1.484,69
2,C_ID_0009150c11,1,2018.000,2018,2018,6,14.000,10,18,2,3.667,3,4,6,2.889,0,6,2,0.333,0,1,-6.570,-0.672,-0.742,-0.730,0,0,0.000,0.000,2018-04-30 13:16:04,2018-03-10 16:03:31,2,1,1.667,9,7,14,1.556,0,0.000,2,0.222,0,0.000,0,0.000,0,0.000,16.111,16,17,0.111,-inf,-inf,-inf,-inf,nan,9,0.000,0,3.667,33.000,0.000,0,-0.559,-0.593,476,425,50,5.556,17


In [36]:
agg_aut_hst_trs_df['aut_elapsed_time_min_purchase'] = (dt.datetime.today() - agg_aut_hst_trs_df['aut_purchase_date_min']).dt.days
agg_aut_hst_trs_df['aut_elapsed_time_max_purchase'] = (dt.datetime.today() - agg_aut_hst_trs_df['aut_purchase_date_max']).dt.days
agg_aut_hst_trs_df['aut_purchase_date_diff'] = (agg_aut_hst_trs_df['aut_purchase_date_max'] - agg_aut_hst_trs_df['aut_purchase_date_min']).dt.days
agg_aut_hst_trs_df['aut_purchase_date_avg'] = agg_aut_hst_trs_df['aut_purchase_date_diff'] / agg_aut_hst_trs_df['aut_card_id_count']
agg_aut_hst_trs_df = pd.merge(agg_aut_hst_trs_df, city_aut_hst_trs_df, on='card_id', how='left')
agg_aut_hst_trs_df.rename(columns={'city_id':'aut_city_id'}, inplace=True)

agg_aut_hst_trs_df.head(3)

,card_id,aut_purchase_year_nunique,aut_purchase_year_mean,aut_purchase_year_min,aut_purchase_year_max,aut_purchase_week_year_nunique,aut_purchase_week_year_mean,aut_purchase_week_year_min,aut_purchase_week_year_max,aut_purchase_month_nunique,aut_purchase_month_mean,aut_purchase_month_min,aut_purchase_month_max,aut_purchase_weekday_nunique,aut_purchase_weekday_mean,aut_purchase_weekday_min,aut_purchase_weekday_max,aut_purchase_weekend_nunique,aut_purchase_weekend_mean,aut_purchase_weekend_min,aut_purchase_weekend_max,aut_purchase_amount_sum,aut_purchase_amount_max,aut_purchase_amount_min,aut_purchase_amount_mean,aut_installments_sum,aut_installments_max,aut_installments_mean,aut_installments_var,aut_purchase_date_max,aut_purchase_date_min,aut_month_lag_max,aut_month_lag_min,aut_month_lag_mean,aut_merchant_id_nunique,aut_merchant_category_id_nunique,aut_near_holidays_sum,aut_near_holidays_mean,aut_near_black_friday_sum,aut_near_black_friday_mean,aut_near_mothers_day_sum,aut_near_mothers_day_mean,aut_near_fathers_day_sum,aut_near_fathers_day_mean,aut_near_childrens_day_sum,aut_near_childrens_day_mean,aut_near_valentines_day_sum,aut_near_valentines_day_mean,aut_purchase_month_diff_mean,aut_purchase_month_diff_min,aut_purchase_month_diff_max,aut_purchase_month_diff_var,aut_price_sum,aut_price_mean,aut_price_max,aut_price_min,aut_price_var,aut_card_id_count,aut_category_1_mean,aut_category_1_sum,aut_category_2_mean,aut_category_2_sum,aut_category_3_mean,aut_category_3_sum,aut_category_2_purchase_mean,aut_category_3_purchase_mean,aut_elapsed_time_min_purchase,aut_elapsed_time_max_purchase,aut_purchase_date_diff,aut_purchase_date_avg,aut_city_id
0,C_ID_0046db9f8a,2,2017.183,2017,2018,40,27.352,1,52,12,6.746,1,12,7,3.155,0,6,2,0.324,0,1,-62.935,3.761,-0.738,-0.443,240,12,1.690,2.953,2018-02-28 10:13:10,2017-01-27 16:36:54,0,-13,-5.056,87,43,161,1.134,10,0.070,14,0.099,11,0.077,9,0.063,11,0.077,16.211,16,17,0.168,-59.316,-0.418,1.039,-0.738,0.126,142,0.021,3,1.576,219.000,1.254,178,-0.603,-0.439,883,486,396,2.789,110
1,C_ID_011b0d9794,2,2017.185,2017,2018,13,31.296,4,52,9,7.630,1,12,6,2.185,0,5,2,0.148,0,1,-5.307,2.425,-0.698,-0.197,121,12,4.481,9.182,2018-02-16 20:23:51,2017-01-30 21:07:06,0,-13,-4.148,7,6,30,1.111,3,0.111,1,0.037,0,0.000,2,0.074,1,0.037,16.370,16,17,0.242,-4.322,-0.160,0.355,-0.698,0.050,27,1.000,27,nan,0.000,1.926,52,nan,-0.059,879,497,381,14.111,-1
2,C_ID_01904d743d,2,2017.100,2017,2018,18,34.980,1,50,8,8.620,1,12,7,2.500,0,6,2,0.180,0,1,-35.392,-0.312,-0.744,-0.708,0,0,0.000,0.000,2018-02-15 13:03:20,2017-07-06 18:21:00,0,-7,-4.180,19,11,41,0.820,10,0.200,0,0.000,5,0.100,8,0.160,0,0.000,16.380,16,17,0.240,-inf,-inf,-inf,-inf,nan,50,0.000,0,1.000,50.000,0.000,0,-0.606,-0.634,723,499,223,4.460,314


In [37]:
agg_rej_hst_trs_df['rej_elapsed_time_min_purchase'] = (dt.datetime.today() - agg_rej_hst_trs_df['rej_purchase_date_min']).dt.days
agg_rej_hst_trs_df['rej_elapsed_time_max_purchase'] = (dt.datetime.today() - agg_rej_hst_trs_df['rej_purchase_date_max']).dt.days
agg_rej_hst_trs_df['rej_purchase_date_diff'] = (agg_rej_hst_trs_df['rej_purchase_date_max'] - agg_rej_hst_trs_df['rej_purchase_date_min']).dt.days
agg_rej_hst_trs_df['rej_purchase_date_avg'] = agg_rej_hst_trs_df['rej_purchase_date_diff'] / agg_rej_hst_trs_df['rej_card_id_count']

agg_rej_hst_trs_df.head(3)


,card_id,rej_purchase_year_nunique,rej_purchase_year_mean,rej_purchase_year_min,rej_purchase_year_max,rej_purchase_week_year_nunique,rej_purchase_week_year_mean,rej_purchase_week_year_min,rej_purchase_week_year_max,rej_purchase_month_nunique,rej_purchase_month_mean,rej_purchase_month_min,rej_purchase_month_max,rej_purchase_weekday_nunique,rej_purchase_weekday_mean,rej_purchase_weekday_min,rej_purchase_weekday_max,rej_purchase_weekend_nunique,rej_purchase_weekend_mean,rej_purchase_weekend_min,rej_purchase_weekend_max,rej_purchase_amount_sum,rej_purchase_amount_max,rej_purchase_amount_min,rej_purchase_amount_mean,rej_installments_sum,rej_installments_max,rej_installments_mean,rej_installments_var,rej_purchase_date_max,rej_purchase_date_min,rej_month_lag_max,rej_month_lag_min,rej_month_lag_mean,rej_merchant_id_nunique,rej_merchant_category_id_nunique,rej_near_holidays_sum,rej_near_holidays_mean,rej_near_black_friday_sum,rej_near_black_friday_mean,rej_near_mothers_day_sum,rej_near_mothers_day_mean,rej_near_fathers_day_sum,rej_near_fathers_day_mean,rej_near_childrens_day_sum,rej_near_childrens_day_mean,rej_near_valentines_day_sum,rej_near_valentines_day_mean,rej_purchase_month_diff_mean,rej_purchase_month_diff_min,rej_purchase_month_diff_max,rej_purchase_month_diff_var,rej_price_sum,rej_price_mean,rej_price_max,rej_price_min,rej_price_var,rej_card_id_count,rej_category_1_mean,rej_category_1_sum,rej_category_2_mean,rej_category_2_sum,rej_category_3_mean,rej_category_3_sum,rej_category_2_purchase_mean,rej_category_3_purchase_mean,rej_elapsed_time_min_purchase,rej_elapsed_time_max_purchase,rej_purchase_date_diff,rej_purchase_date_avg
0,C_ID_0046db9f8a,2,2017.500,2017.000,2018.000,4,25.125,2.000,51.000,3,6.250,1.000,12.000,4,1.875,0.000,5.000,2,0.125,0.000,1.000,1.175,3.625,-0.705,0.147,15,6.000,1.875,5.554,2018-01-11 13:24:36,2017-11-23 00:00:00,-1.000,-3.000,-1.750,5,5,4,0.500,2,0.250,0,0.000,0,0.000,0,0.000,0,0.000,16.125,16.000,17.000,0.125,-0.818,-0.102,0.604,-0.705,0.180,8,0.000,0,1.000,8.000,1.250,10,9.817,4.030,583.000,534.000,49.000,6.125
1,C_ID_011b0d9794,0,nan,nan,nan,0,nan,nan,nan,0,nan,nan,nan,0,nan,nan,nan,0,nan,nan,nan,0.000,nan,nan,nan,0,nan,nan,nan,NaT,NaT,nan,nan,nan,0,0,0,nan,0,nan,0,nan,0,nan,0,nan,0,nan,nan,nan,nan,nan,0.000,nan,nan,nan,nan,0,nan,0,nan,0.000,nan,0,nan,nan,nan,nan,nan,nan
2,C_ID_01904d743d,2,2017.143,2017.000,2018.000,9,35.429,5.000,52.000,6,8.524,2.000,12.000,5,3.048,1.000,6.000,2,0.095,0.000,1.000,-14.954,-0.546,-0.740,-0.712,1,1.000,0.048,0.048,2018-02-10 07:53:44,2017-07-21 18:04:23,0.000,-7.000,-3.762,6,6,27,1.286,5,0.238,0,0.000,6,0.286,0,0.000,0,0.000,16.190,16.000,17.000,0.162,-inf,-inf,-0.703,-inf,nan,21,0.000,0,1.000,21.000,0.048,1,9.817,12.522,708.000,504.000,203.000,9.667


## Merge new columns with training and test datasets

In [38]:
#Train
train_df = pd.merge(train_df, agg_new_trs_df, on='card_id', how='left')
train_df = pd.merge(train_df, agg_aut_hst_trs_df, on='card_id', how='left')
train_df = pd.merge(train_df, agg_rej_hst_trs_df, on='card_id', how='left')

train_df.head(3)

,card_id,feature_1,feature_2,feature_3,target,first_active_year,first_active_months,elapsed_time,first_active_quarter,new_purchase_year_nunique,new_purchase_year_mean,new_purchase_year_min,new_purchase_year_max,new_purchase_week_year_nunique,new_purchase_week_year_mean,new_purchase_week_year_min,new_purchase_week_year_max,new_purchase_month_nunique,new_purchase_month_mean,new_purchase_month_min,new_purchase_month_max,new_purchase_weekday_nunique,new_purchase_weekday_mean,new_purchase_weekday_min,new_purchase_weekday_max,new_purchase_weekend_nunique,new_purchase_weekend_mean,new_purchase_weekend_min,new_purchase_weekend_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_purchase_date_max,new_purchase_date_min,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_near_holidays_sum,new_near_holidays_mean,new_near_black_friday_sum,new_near_black_friday_mean,new_near_mothers_day_sum,new_near_mothers_day_mean,...,rej_purchase_weekend_max,rej_purchase_amount_sum,rej_purchase_amount_max,rej_purchase_amount_min,rej_purchase_amount_mean,rej_installments_sum,rej_installments_max,rej_installments_mean,rej_installments_var,rej_purchase_date_max,rej_purchase_date_min,rej_month_lag_max,rej_month_lag_min,rej_month_lag_mean,rej_merchant_id_nunique,rej_merchant_category_id_nunique,rej_near_holidays_sum,rej_near_holidays_mean,rej_near_black_friday_sum,rej_near_black_friday_mean,rej_near_mothers_day_sum,rej_near_mothers_day_mean,rej_near_fathers_day_sum,rej_near_fathers_day_mean,rej_near_childrens_day_sum,rej_near_childrens_day_mean,rej_near_valentines_day_sum,rej_near_valentines_day_mean,rej_purchase_month_diff_mean,rej_purchase_month_diff_min,rej_purchase_month_diff_max,rej_purchase_month_diff_var,rej_price_sum,rej_price_mean,rej_price_max,rej_price_min,rej_price_var,rej_card_id_count,rej_category_1_mean,rej_category_1_sum,rej_category_2_mean,rej_category_2_sum,rej_category_3_mean,rej_category_3_sum,rej_category_2_purchase_mean,rej_category_3_purchase_mean,rej_elapsed_time_min_purchase,rej_elapsed_time_max_purchase,rej_purchase_date_diff,rej_purchase_date_avg
0,C_ID_92a2005557,5,2,1,-0.820,2017,6,758,2,1.000,2018.000,2018.000,2018.000,7.000,13.304,10.000,17.000,2.000,3.478,3.000,4.000,7.000,3.130,0.000,6.000,2.000,0.261,0.000,1.000,-13.244,-0.296,-0.724,-0.576,0.000,0.000,0.000,0.000,2018-04-29 11:23:05,2018-03-05 14:04:36,2.000,1.000,1.478,23.000,14.000,34.000,1.478,0.000,0.000,6.000,0.261,...,1.000,-8.572,-0.432,-0.738,-0.659,4,1.000,0.308,0.231,2017-12-27 14:33:33,2017-07-15 14:58:10,-2.000,-7.000,-4.462,12,10,15,1.154,2,0.154,0,0.000,3,0.231,3,0.231,0,0.000,16.308,16.000,17.000,0.231,-inf,-inf,-0.616,-inf,nan,13,0.000,0,1.000,13.000,0.308,4,9.817,9.540,714.000,549.000,164.000,12.615
1,C_ID_3d0044924f,4,1,0,0.393,2017,1,909,1,1.000,2018.000,2018.000,2018.000,4.000,9.000,5.000,13.000,2.000,2.500,2.000,3.000,4.000,1.500,0.000,4.000,1.000,0.000,0.000,0.000,-4.356,-0.702,-0.739,-0.726,6.000,1.000,1.000,0.000,2018-03-30 06:48:26,2018-02-01 17:07:54,2.000,1.000,1.500,6.000,5.000,9.000,1.500,0.000,0.000,0.000,0.000,...,1.000,-1.123,1.943,-0.741,-0.102,42,10.000,3.818,12.164,2017-12-28 13:21:18,2017-03-03 21:27:17,-1.000,-10.000,-4.455,9,9,9,0.818,0,0.000,1,0.091,1,0.091,5,0.455,0,0.000,17.364,17.000,18.000,0.255,-2.869,-0.261,0.194,-0.741,0.125,11,0.182,2,1.000,9.000,1.636,18,9.817,1.044,847.000,548.000,299.000,27.182
2,C_ID_d639edf6cd,2,2,0,0.688,2016,8,1062,3,1.000,2018.000,2018.000,2018.000,1.000,17.000,17.000,17.000,1.000,4.000,4.000,4.000,1.000,5.000,5.000,5.000,1.000,1.000,1.000,1.000,-0.700,-0.700,-0.700,-0.700,0.000,0.000,0.000,nan,2018-04-28 17:43:11,2018-04-28 17:43:11,2.000,2.000,2.000,1.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,...,0.000,-1.339,-0.638,-0.701,-0.669,0,0.000,0.000,0.000,2017-04-21 19:00:14,2017-02-23 19:32:09,-10.000,-12

In [39]:
#Test
test_df = pd.merge(test_df, agg_new_trs_df, on='card_id', how='left')
test_df = pd.merge(test_df, agg_aut_hst_trs_df, on='card_id', how='left')
test_df = pd.merge(test_df, agg_rej_hst_trs_df, on='card_id', how='left')

test_df.head(3)

,card_id,feature_1,feature_2,feature_3,first_active_year,first_active_months,elapsed_time,first_active_quarter,new_purchase_year_nunique,new_purchase_year_mean,new_purchase_year_min,new_purchase_year_max,new_purchase_week_year_nunique,new_purchase_week_year_mean,new_purchase_week_year_min,new_purchase_week_year_max,new_purchase_month_nunique,new_purchase_month_mean,new_purchase_month_min,new_purchase_month_max,new_purchase_weekday_nunique,new_purchase_weekday_mean,new_purchase_weekday_min,new_purchase_weekday_max,new_purchase_weekend_nunique,new_purchase_weekend_mean,new_purchase_weekend_min,new_purchase_weekend_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_purchase_date_max,new_purchase_date_min,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_near_holidays_sum,new_near_holidays_mean,new_near_black_friday_sum,new_near_black_friday_mean,new_near_mothers_day_sum,new_near_mothers_day_mean,new_near_fathers_day_sum,...,rej_purchase_weekend_max,rej_purchase_amount_sum,rej_purchase_amount_max,rej_purchase_amount_min,rej_purchase_amount_mean,rej_installments_sum,rej_installments_max,rej_installments_mean,rej_installments_var,rej_purchase_date_max,rej_purchase_date_min,rej_month_lag_max,rej_month_lag_min,rej_month_lag_mean,rej_merchant_id_nunique,rej_merchant_category_id_nunique,rej_near_holidays_sum,rej_near_holidays_mean,rej_near_black_friday_sum,rej_near_black_friday_mean,rej_near_mothers_day_sum,rej_near_mothers_day_mean,rej_near_fathers_day_sum,rej_near_fathers_day_mean,rej_near_childrens_day_sum,rej_near_childrens_day_mean,rej_near_valentines_day_sum,rej_near_valentines_day_mean,rej_purchase_month_diff_mean,rej_purchase_month_diff_min,rej_purchase_month_diff_max,rej_purchase_month_diff_var,rej_price_sum,rej_price_mean,rej_price_max,rej_price_min,rej_price_var,rej_card_id_count,rej_category_1_mean,rej_category_1_sum,rej_category_2_mean,rej_category_2_sum,rej_category_3_mean,rej_category_3_sum,rej_category_2_purchase_mean,rej_category_3_purchase_mean,rej_elapsed_time_min_purchase,rej_elapsed_time_max_purchase,rej_purchase_date_diff,rej_purchase_date_avg
0,C_ID_0ab67a22ab,3,3,1,2017,4,819,2,1.000,2018.000,2018.000,2018.000,3.000,7.000,5.000,9.000,1.000,2.000,2.000,2.000,3.000,3.667,2.000,5.000,2.000,0.333,0.000,1.000,-1.777,-0.383,-0.722,-0.592,5.000,3.000,1.667,1.333,2018-02-28 19:08:52,2018-02-03 09:44:29,2.000,2.000,2.000,3.000,3.000,3.000,1.000,0.000,0.000,0.000,0.000,0.000,...,1.000,-12.695,0.060,-0.729,-0.529,78,12.000,3.250,7.674,2017-12-13 17:19:04,2017-04-27 14:17:27,0.000,-8.000,-3.917,7,7,24,1.000,3,0.125,1,0.042,1,0.042,8,0.333,3,0.125,18.250,18.000,19.000,0.196,-9.039,-0.377,0.005,-0.729,0.095,24,0.542,13,1.000,11.000,1.542,37,9.817,1.127,793.000,563.000,230.000,9.583
1,C_ID_130fd0cbdd,2,3,0,2017,1,909,1,1.000,2018.000,2018.000,2018.000,6.000,12.444,9.000,16.000,2.000,3.444,3.000,4.000,6.000,3.000,0.000,6.000,2.000,0.333,0.000,1.000,-5.945,-0.506,-0.741,-0.661,11.000,3.000,1.222,0.444,2018-04-20 18:00:17,2018-03-03 12:18:48,2.000,1.000,1.444,9.000,8.000,11.000,1.222,0.000,0.000,3.000,0.333,0.000,...,1.000,-0.732,-0.732,-0.732,-0.732,1,1.000,1.000,nan,2017-05-06 13:51:42,2017-05-06 13:51:42,-9.000,-9.000,-9.000,1,1,0,0.000,0,0.000,1,1.000,0,0.000,0,0.000,0,0.000,17.000,17.000,17.000,nan,-0.732,-0.732,-0.732,-0.732,nan,1,1.000,1,nan,0.000,1.000,1,nan,1.603,784.000,784.000,0.000,0.000
2,C_ID_b709037bc5,5,1,1,2017,8,697,3,1.000,2018.000,2018.000,2018.000,2.000,10.000,9.000,11.000,1.000,3.000,3.000,3.000,2.000,2.000,1.000,3.000,1.000,0.000,0.000,0.000,0.180,0.905,-0.724,0.090,11.000,10.000,5.500,40.500,2018-03-13 13:10:46,2018-03-01 14:51:33,1.000,1.000,1.000,2.000,2.000,2.000,1.000,0.000,0.000,0.000,0.000,0.000,...,0.000,5.423,2.526,0.215,1.356,18,10.000,4.500,40.333,2018-02-02 17:36:11,2018-01-11 14:27:59,0.000,-1.

## Delete columns that will not be used

In [40]:
drop_cols = ['new_purchase_date_max',
             'new_purchase_date_min',
             'aut_purchase_date_max',
             'aut_purchase_date_min',
             'rej_purchase_date_max',
             'rej_purchase_date_min']

train_df.drop(drop_cols, axis=1, inplace=True)
test_df.drop(drop_cols, axis=1, inplace=True)

## Creates a column to inform if the line has NA

In [41]:
train_df["is_na"] = pd.isnull(train_df).any(axis=1).map({True: 1, False: 0}).astype(int)
test_df["is_na"] = pd.isnull(test_df).any(axis=1).map({True: 1, False: 0}).astype(int)

In [42]:
print("Train shape: ", train_df.shape)
print("Test shape: ", test_df.shape)

Train shape:  (201917, 213)
Test shape:  (123623, 212)


In [43]:
train_df.head()

,card_id,feature_1,feature_2,feature_3,target,first_active_year,first_active_months,elapsed_time,first_active_quarter,new_purchase_year_nunique,new_purchase_year_mean,new_purchase_year_min,new_purchase_year_max,new_purchase_week_year_nunique,new_purchase_week_year_mean,new_purchase_week_year_min,new_purchase_week_year_max,new_purchase_month_nunique,new_purchase_month_mean,new_purchase_month_min,new_purchase_month_max,new_purchase_weekday_nunique,new_purchase_weekday_mean,new_purchase_weekday_min,new_purchase_weekday_max,new_purchase_weekend_nunique,new_purchase_weekend_mean,new_purchase_weekend_min,new_purchase_weekend_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_near_holidays_sum,new_near_holidays_mean,new_near_black_friday_sum,new_near_black_friday_mean,new_near_mothers_day_sum,new_near_mothers_day_mean,new_near_fathers_day_sum,new_near_fathers_day_mean,...,rej_purchase_weekend_min,rej_purchase_weekend_max,rej_purchase_amount_sum,rej_purchase_amount_max,rej_purchase_amount_min,rej_purchase_amount_mean,rej_installments_sum,rej_installments_max,rej_installments_mean,rej_installments_var,rej_month_lag_max,rej_month_lag_min,rej_month_lag_mean,rej_merchant_id_nunique,rej_merchant_category_id_nunique,rej_near_holidays_sum,rej_near_holidays_mean,rej_near_black_friday_sum,rej_near_black_friday_mean,rej_near_mothers_day_sum,rej_near_mothers_day_mean,rej_near_fathers_day_sum,rej_near_fathers_day_mean,rej_near_childrens_day_sum,rej_near_childrens_day_mean,rej_near_valentines_day_sum,rej_near_valentines_day_mean,rej_purchase_month_diff_mean,rej_purchase_month_diff_min,rej_purchase_month_diff_max,rej_purchase_month_diff_var,rej_price_sum,rej_price_mean,rej_price_max,rej_price_min,rej_price_var,rej_card_id_count,rej_category_1_mean,rej_category_1_sum,rej_category_2_mean,rej_category_2_sum,rej_category_3_mean,rej_category_3_sum,rej_category_2_purchase_mean,rej_category_3_purchase_mean,rej_elapsed_time_min_purchase,rej_elapsed_time_max_purchase,rej_purchase_date_diff,rej_purchase_date_avg,is_na
0,C_ID_92a2005557,5,2,1,-0.820,2017,6,758,2,1.000,2018.000,2018.000,2018.000,7.000,13.304,10.000,17.000,2.000,3.478,3.000,4.000,7.000,3.130,0.000,6.000,2.000,0.261,0.000,1.000,-13.244,-0.296,-0.724,-0.576,0.000,0.000,0.000,0.000,2.000,1.000,1.478,23.000,14.000,34.000,1.478,0.000,0.000,6.000,0.261,0.000,0.000,...,0.000,1.000,-8.572,-0.432,-0.738,-0.659,4,1.000,0.308,0.231,-2.000,-7.000,-4.462,12,10,15,1.154,2,0.154,0,0.000,3,0.231,3,0.231,0,0.000,16.308,16.000,17.000,0.231,-inf,-inf,-0.616,-inf,nan,13,0.000,0,1.000,13.000,0.308,4,9.817,9.540,714.000,549.000,164.000,12.615,1
1,C_ID_3d0044924f,4,1,0,0.393,2017,1,909,1,1.000,2018.000,2018.000,2018.000,4.000,9.000,5.000,13.000,2.000,2.500,2.000,3.000,4.000,1.500,0.000,4.000,1.000,0.000,0.000,0.000,-4.356,-0.702,-0.739,-0.726,6.000,1.000,1.000,0.000,2.000,1.000,1.500,6.000,5.000,9.000,1.500,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,1.000,-1.123,1.943,-0.741,-0.102,42,10.000,3.818,12.164,-1.000,-10.000,-4.455,9,9,9,0.818,0,0.000,1,0.091,1,0.091,5,0.455,0,0.000,17.364,17.000,18.000,0.255,-2.869,-0.261,0.194,-0.741,0.125,11,0.182,2,1.000,9.000,1.636,18,9.817,1.044,847.000,548.000,299.000,27.182,0
2,C_ID_d639edf6cd,2,2,0,0.688,2016,8,1062,3,1.000,2018.000,2018.000,2018.000,1.000,17.000,17.000,17.000,1.000,4.000,4.000,4.000,1.000,5.000,5.000,5.000,1.000,1.000,1.000,1.000,-0.700,-0.700,-0.700,-0.700,0.000,0.000,0.000,nan,2.000,2.000,2.000,1.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,0.000,0.000,...,0.000,0.000,-1.339,-0.638,-0.701,-0.669,0,0.000,0.000,0.000,-10.000,-12.000,-11.000,1,1,3,1.500,0,0.000,1,0.500,0,0.000,0,0.000,0,0.000,16.000,16.000,16.000,0.000,-inf,-inf,-inf,-inf,nan,2,0.000,0,5.000,10.000,0.000,0,6.764,13.068,855.000,799.000,56.000,28.00

In [44]:
test_df.head()

,card_id,feature_1,feature_2,feature_3,first_active_year,first_active_months,elapsed_time,first_active_quarter,new_purchase_year_nunique,new_purchase_year_mean,new_purchase_year_min,new_purchase_year_max,new_purchase_week_year_nunique,new_purchase_week_year_mean,new_purchase_week_year_min,new_purchase_week_year_max,new_purchase_month_nunique,new_purchase_month_mean,new_purchase_month_min,new_purchase_month_max,new_purchase_weekday_nunique,new_purchase_weekday_mean,new_purchase_weekday_min,new_purchase_weekday_max,new_purchase_weekend_nunique,new_purchase_weekend_mean,new_purchase_weekend_min,new_purchase_weekend_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_near_holidays_sum,new_near_holidays_mean,new_near_black_friday_sum,new_near_black_friday_mean,new_near_mothers_day_sum,new_near_mothers_day_mean,new_near_fathers_day_sum,new_near_fathers_day_mean,new_near_childrens_day_sum,...,rej_purchase_weekend_min,rej_purchase_weekend_max,rej_purchase_amount_sum,rej_purchase_amount_max,rej_purchase_amount_min,rej_purchase_amount_mean,rej_installments_sum,rej_installments_max,rej_installments_mean,rej_installments_var,rej_month_lag_max,rej_month_lag_min,rej_month_lag_mean,rej_merchant_id_nunique,rej_merchant_category_id_nunique,rej_near_holidays_sum,rej_near_holidays_mean,rej_near_black_friday_sum,rej_near_black_friday_mean,rej_near_mothers_day_sum,rej_near_mothers_day_mean,rej_near_fathers_day_sum,rej_near_fathers_day_mean,rej_near_childrens_day_sum,rej_near_childrens_day_mean,rej_near_valentines_day_sum,rej_near_valentines_day_mean,rej_purchase_month_diff_mean,rej_purchase_month_diff_min,rej_purchase_month_diff_max,rej_purchase_month_diff_var,rej_price_sum,rej_price_mean,rej_price_max,rej_price_min,rej_price_var,rej_card_id_count,rej_category_1_mean,rej_category_1_sum,rej_category_2_mean,rej_category_2_sum,rej_category_3_mean,rej_category_3_sum,rej_category_2_purchase_mean,rej_category_3_purchase_mean,rej_elapsed_time_min_purchase,rej_elapsed_time_max_purchase,rej_purchase_date_diff,rej_purchase_date_avg,is_na
0,C_ID_0ab67a22ab,3,3,1,2017,4,819,2,1.000,2018.000,2018.000,2018.000,3.000,7.000,5.000,9.000,1.000,2.000,2.000,2.000,3.000,3.667,2.000,5.000,2.000,0.333,0.000,1.000,-1.777,-0.383,-0.722,-0.592,5.000,3.000,1.667,1.333,2.000,2.000,2.000,3.000,3.000,3.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,1.000,-12.695,0.060,-0.729,-0.529,78,12.000,3.250,7.674,0.000,-8.000,-3.917,7,7,24,1.000,3,0.125,1,0.042,1,0.042,8,0.333,3,0.125,18.250,18.000,19.000,0.196,-9.039,-0.377,0.005,-0.729,0.095,24,0.542,13,1.000,11.000,1.542,37,9.817,1.127,793.000,563.000,230.000,9.583,0
1,C_ID_130fd0cbdd,2,3,0,2017,1,909,1,1.000,2018.000,2018.000,2018.000,6.000,12.444,9.000,16.000,2.000,3.444,3.000,4.000,6.000,3.000,0.000,6.000,2.000,0.333,0.000,1.000,-5.945,-0.506,-0.741,-0.661,11.000,3.000,1.222,0.444,2.000,1.000,1.444,9.000,8.000,11.000,1.222,0.000,0.000,3.000,0.333,0.000,0.000,0.000,...,1.000,1.000,-0.732,-0.732,-0.732,-0.732,1,1.000,1.000,nan,-9.000,-9.000,-9.000,1,1,0,0.000,0,0.000,1,1.000,0,0.000,0,0.000,0,0.000,17.000,17.000,17.000,nan,-0.732,-0.732,-0.732,-0.732,nan,1,1.000,1,nan,0.000,1.000,1,nan,1.603,784.000,784.000,0.000,0.000,1
2,C_ID_b709037bc5,5,1,1,2017,8,697,3,1.000,2018.000,2018.000,2018.000,2.000,10.000,9.000,11.000,1.000,3.000,3.000,3.000,2.000,2.000,1.000,3.000,1.000,0.000,0.000,0.000,0.180,0.905,-0.724,0.090,11.000,10.000,5.500,40.500,1.000,1.000,1.000,2.000,2.000,2.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.423,2.526,0.215,1.356,18,10.000,4.500,40.333,0.000,-1.000,-0.500,2,2,4,1.000,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000,16.500,16.000,17.000,0.333,-0.987,-0.247,0.253,-1.176,0.424,4,0.000,0,5.000,20.000,1.000,4,6.764,6.896,534.000,512.000,22.0

## Generates files with the columns that will be used in the training of the model

In [45]:
train_df.to_csv("train_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)